In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import sys
import requests
import urllib
import time
import datetime

In [2]:
# Use the following site to get list of all publicly traded Stocks
#http://nxcoreapi.com/symbols.php

#nyse
nyse = pd.read_html('http://nxcoreapi.com/symbols.php?search=&m_exchange=NYSE&m_type=&s_type=exact&m_symbol=on&m_name=on')
#nasdaq
nqnm = pd.read_html('http://nxcoreapi.com/symbols.php?search=&m_exchange=NQNM&m_type=&s_type=exact&m_symbol=on&m_name=on')
#amex
amex = pd.read_html('http://nxcoreapi.com/symbols.php?search=&m_exchange=AMEX&m_type=&s_type=exact&m_symbol=on&m_name=on')

In [3]:
nyse = nyse[0]
nyse.columns = nyse.iloc[0]
nyse = nyse[1:]
# nyse

In [4]:
nqnm = nqnm[0]
nqnm.columns = nqnm.iloc[0]
nqnm = nqnm[1:]
# nqnm

In [5]:
amex = amex[0]
amex.columns = amex.iloc[0]
amex = amex[1:]
# amex

In [6]:
stocks = nyse.copy()
stocks = stocks.append(nqnm)
stocks = stocks.append(amex)
# stocks.tail()
# stocks


In [7]:
stocks = stocks[['Symbol', 'Name', 'Exchange']]
stocks.reset_index(inplace=True)
stocks = stocks.drop(columns=['index'])
stocks

,Symbol,Name,Exchange
0,A,"Agilent Technologies, Inc",NYSE
1,AA,Alcoa Corp,NYSE
2,AAN,"Aaron's, Inc",NYSE
3,AAP,"Advance Auto Parts, Inc W/I",NYSE
4,AAT,"American Assets Trust, Inc",NYSE
...,...,...,...
6276,YCBD,"cbdMD, Inc.",AMEX
6277,YCBD.PR.A,"cbdMD, Inc. 8.0% Series A Cumulative Convertib...",AMEX
6278,YUMA,"Yuma Energy, Inc",AMEX
6279,ZDGE,"Zedge, Inc",AMEX


In [8]:
features = ['industry', 'sector', 'dividendRate', 'exDividendDate', 'lastSplitDate', 'lastSplitFactor']
    
for feature in features:
    stocks.loc[:, feature] = np.nan
    
stocks[features] = stocks[features].astype('object')


In [9]:
# symbol = yf.Ticker('A')
# symbol.info
# symbol.quarterly_financials

In [10]:
# uncomment the following when debugging
# stocks = stocks[0:5]
# stocks

In [11]:
def getInfo(symbol, info=True, calendar=True, quarterly_financials=True):
    connection_tries=0
    j=0
    
    # these booleans are used so that we don't get the same data again if a connection error happens afterwards
    info_bool = False
    calendar_bool = False
    quarterly_bool = False
    
    info = {}
    calendar = {}
    quarterly_financials = {}
    
    while 1:
        
        try:
            if info_bool == False:
                info = symbol.info
                info_bool = True
                
            if calendar_bool == False:
                calendar = symbol.calendar
                calendar_bool = True
                
            if quarterly_bool == False:
                quarterly_financials = symbol.quarterly_financials
                quarterly_bool = True
                
            return info, calendar, quarterly_financials
            
        except (urllib.error.HTTPError, requests.exceptions.ConnectionError) as e:
            
            if connection_tries < 1:
                print (e, "HTTP Error. Will try again in 10 seconds")
                time.sleep(10)
                connection_tries +=1
            elif connection_tries < 2:
                print (e, "HTTP Error. Will try again in 30 seconds")
                time.sleep(30)
                connection_tries +=1
            elif connection_tries < 4:
                print (e, "HTTP Error. Will try again in 5 minutes")
                time.sleep(300)
                connection_tries +=1
            elif connection_tries < 5:
                print (e, "HTTP Error.  Will try again in 1 hour")
                time.sleep(3600)
                connection_tries +=1
            else:
                connection_tries = 0
                print (e, "HTTP Error.  Giving up.")
                return {}, pd.DataFrame(), pd.DataFrame()
        except:
            e = sys.exc_info()[0]
#             print('here: ', e)
            if j>2:
                return 'error', 'error', 'error'
            j+=1
            

In [12]:
i = 0
initial_time = round(time.time())
stocks.head()

# stocks[features] = stocks[features].astype('object')

bad_symbols = []
symbols_with_no_return = []

for row in stocks.itertuples():
    print (i)    
    if i % 100 == 0:
        print ("It has been: ", (round(time.time())-initial_time)/ 60.0, " minutes")
        
    if '/' in row[1]:
        print ('Bad symbol in ', row[1])
        bad_symbols.append(i)
    elif '.' in row[1]:
        print ('Bad symbol in ', row[1])
        bad_symbols.append(i)
    else:
        
        symbol = row[1]
        symbol = yf.Ticker(symbol)           

        info, calendar, quarterly_financials = getInfo(symbol)
        
        if info == 'error':
            symbols_with_no_return.append(row[1])
            print('No return: ', row[1])
        
        else:
        
            for feature in features:

                #try twice in case there's a fail that shouldn't happen

                try:
                    value = info[feature]
                    stocks.at[i, feature] = value

                except:
                    stocks.at[i, feature] = np.nan
                    e = sys.exc_info()[0]
                    print(e, ": ", feature)

        #     get any upcoming earnings date
            try:       
                if 'Value' in calendar.columns:
                    column = 'Value'
                else:
                    column = 0
                value = calendar[column]['Earnings Date']
                stocks.at[i, 'upcoming_earnings_date'] = value
            except:
                stocks.at[i, 'upcoming_earnings_date'] = np.nan

            #get the previous quarterly earnings date
            try:
                value = quarterly_financials.columns[0]
                stocks.at[i, 'previous_earnings_date'] = value
            except:
                stocks.at[i, 'previous_earnings_date'] = np.nan
    
    i += 1
    

0
It has been:  0.0  minutes
1
2
3
4
5
6
7
8
9
10
11
12
13
Bad symbol in  ABR.PR.A
14
Bad symbol in  ABR.PR.B
15
Bad symbol in  ABR.PR.C
16
17
18
19
20
No return:  ACC
21
22
23
Bad symbol in  ACEL.WS
24
25
26
27
28
Bad symbol in  ACND.U
29
No return:  ACP
30
31
No return:  ACV
32
33
No return:  ADCT
34
35
36
37
38
39
40
No return:  AEB
41
No return:  AEE
42
No return:  AEFC
43
44
No return:  AEL
45
Bad symbol in  AEL.PR.A
46
Bad symbol in  AEL.PR.B
47
48
49
50
Bad symbol in  AEP.PR.B
51
Bad symbol in  AEP.PR.C
52
53
54
55
No return:  AFC
56
57
No return:  AFGB
58
No return:  AFGC
59
60
No return:  AFGH
61
No return:  AFI
62
63
64
65
66
67
68
69
Bad symbol in  AGM.A
70
Bad symbol in  AGM.PR.A
71
Bad symbol in  AGM.PR.C
72
Bad symbol in  AGM.PR.D
73
Bad symbol in  AGM.PR.E
74
Bad symbol in  AGM.PR.F
75
No return:  AGN
76
77
Bad symbol in  AGO.PR.B
78
Bad symbol in  AGO.PR.E
79
Bad symbol in  AGO.PR.F
80
No return:  AGR
81
82
83
84
85
86
Bad symbol in  AHH.PR.A
87
Bad symbol in  AHL.PR.C


No return:  CEM
594
No return:  CEN
595
596
No return:  CEPU
597
598
Bad symbol in  CEQP.PR
599
600
It has been:  34.53333333333333  minutes
601
Bad symbol in  CFG.PR.D
602
Bad symbol in  CFG.PR.E
603
604
Bad symbol in  CFR.PR.A
605
Bad symbol in  CFR.PR.A/CL
606
607
No return:  CFXA
608
609
610
611
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
<class 'KeyError'> :  lastSplitDate
<class 'KeyError'> :  lastSplitFactor
612
No return:  CHCT
613
614
615
616
617
No return:  CHK
618
Bad symbol in  CHK.PR.D
619
620
621
622
Bad symbol in  CHMI.PR.A
623
Bad symbol in  CHMI.PR.B
624
625
626
627
628
629
630
631
Bad symbol in  CHX/WI
632
633
634
635
636
No return:  CIF
637
638
Bad symbol in  CIG.C
639
640
641
Bad symbol in  CIM.PR.A
642
Bad symbol in  CIM.PR.B
643
Bad symbol in  CIM.PR.C
644
Bad symbol in  CIM.PR.D
645
646
647
Bad symbol in  CIO.PR.A
648
649
650
Bad symbol in  CIT.PR.B
651
652
653
Bad symbol in  CLA.U
654
655
No return:  CLDR
656
657
658
No return:  CLGX
659
660
661


No return:  FDP
1142
1143
1144
No return:  FE
1145
No return:  FEAC
1146
Bad symbol in  FEAC.U
1147
Bad symbol in  FEAC.WS
1148
1149
No return:  FEI
1150
1151
1152
1153
1154
1155
1156
No return:  FFG
1157
No return:  FG
1158
Bad symbol in  FG.WS
1159
No return:  FGB
1160
1161
1162
Bad symbol in  FHN.PR.A
1163
Bad symbol in  FHN.PR.B
1164
Bad symbol in  FHN.PR.C
1165
Bad symbol in  FHN.PR.D
1166
Bad symbol in  FHN.PR.E
1167
1168
1169
No return:  FIF
1170
1171
1172
1173
1174
No return:  FIV
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
No return:  FMY
1191
1192
1193
Bad symbol in  FNB.PR.E
1194
1195
1196
1197
1198
No return:  FOF
1199
1200
It has been:  76.5  minutes
1201
1202
Bad symbol in  FPAC.U
1203
Bad symbol in  FPAC.WS
1204
No return:  FPF
1205
No return:  FPH
1206
1207
Bad symbol in  FPI.PR.B
1208
No return:  FPL
1209
1210
1211
1212
Bad symbol in  FRC.PR.F
1213
Bad symbol in  FRC.PR.G
1214
Bad symbol in  FRC.PR.H
1215
Bad symbol in  FRC.PR.I
1216

1684
No return:  JPT
1685
1686
No return:  JRI
1687
No return:  JRO
1688
1689
No return:  JSD
1690
No return:  JT
1691
1692
1693
Bad symbol in  JW.A
1694
Bad symbol in  JW.B
1695
1696
No return:  JWS
1697
Bad symbol in  JWS.U
1698
Bad symbol in  JWS.WS
1699
1700
It has been:  111.98333333333333  minutes
1701
1702
1703
1704
1705
1706
No return:  KCAC
1707
Bad symbol in  KCAC.U
1708
Bad symbol in  KCAC.WS
1709
1710
No return:  KDP
1711
No return:  KEM
1712
No return:  KEN
1713
1714
1715
1716
Bad symbol in  KEY.PR.I
1717
Bad symbol in  KEY.PR.J
1718
Bad symbol in  KEY.PR.K
1719
No return:  KEYS
1720
No return:  KF
1721
1722
1723
1724
1725
Bad symbol in  KIM.PR.L
1726
Bad symbol in  KIM.PR.M
1727
No return:  KIO
1728
1729
Bad symbol in  KKR.PR.A
1730
Bad symbol in  KKR.PR.B
1731
Bad symbol in  KKR.PR.C
1732
1733
1734
No return:  KMF
1735
1736
1737
1738
1739
1740
1741
No return:  KNOP
1742
1743
1744
1745
1746
1747
1748
No return:  KR
1749
No return:  KRA
1750
1751
1752
1753
1754
1755
No ret

2283
2284
No return:  OIA
2285
Bad symbol in  OIBR.C
2286
2287
No return:  OIS
2288
2289
2290
2291
No return:  OMC
2292
2293
2294
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
<class 'KeyError'> :  lastSplitDate
<class 'KeyError'> :  lastSplitFactor
2295
2296
2297
2298
2299
No return:  OPP
2300
It has been:  154.75  minutes
2301
2302
2303
2304
2305
2306
2307
No return:  ORI
2308
No return:  ORN
2309
2310
2311
2312
No return:  OSK
2313
Bad symbol in  OSLE/CL
2314
2315
Bad symbol in  OTIS/WI
2316
2317
2318
No return:  OXM
2319
2320
Bad symbol in  OXY.WS
2321
Bad symbol in  OXY.WS/WI
2322
2323
No return:  PAC
2324
2325
2326
Bad symbol in  PACK.WS
2327
2328
2329
2330
No return:  PAI
2331
2332
No return:  PANA
2333
Bad symbol in  PANA.U
2334
Bad symbol in  PANA.WS
2335
2336
2337
2338
No return:  PAYC
2339
No return:  PB
2340
2341
No return:  PBB
2342
No return:  PBC
2343
2344
2345
2346
2347
Bad symbol in  PBI.PR.B
2348
2349
Bad symbol in  PBR.A
2350
2351
No return:  PBY
2352
2

...

<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4174
4175
4176
HTTP Error 503: Service Unavailable HTTP Error. Will try again in 10 seconds
No return:  FCEF
4177
4178
4179
4180
No return:  FCNCP
4181
4182
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4183
4184
No return:  FDEF
4185
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4186
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4187
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4188
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4189
4190
No return:  FDUSG
4191
No return:  FDUSL
4192
No return:  FDUSZ
4193
4194
4195
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4196
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4197
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4198
No return:  FEP
4199
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4200
It has been:  287.56666666666666  minutes
<class 'KeyError'> :  in

<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4526
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4527
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4528
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4529
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4530
No return:  IBTK
4531
4532
No return:  IBUY
4533
4534
4535
4536
4537
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4538
4539
4540
No return:  ICON
4541
4542
4543
4544
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4545
4546
4547
4548
4549
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4550
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4551
4552
4553
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4554
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4555
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
4556
4557
No return:  IFV
4558
<class 'KeyError'> :  industry
<class 'KeyError'

No return:  NXGN
5022
No return:  NXPI
5023
5024
5025
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5026
5027
5028
5029
5030
No return:  NYMTP
5031
5032
5033
5034
5035
No return:  OCC
5036
5037
5038
5039
5040
No return:  OCSLL
5041
5042
5043
5044
5045
5046
5047
No return:  OFS
5048
No return:  OFSSI
5049
No return:  OFSSL
5050
No return:  OFSSZ
5051
5052
5053
5054
No return:  OLD
5055
5056
5057
No return:  OMAB
5058
No return:  OMCL
5059
5060
5061
5062
5063
5064
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5065
5066
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
<class 'KeyError'> :  lastSplitDate
<class 'KeyError'> :  lastSplitFactor
5067
No return:  OPBK
5068
5069
5070
5071
No return:  OPINL
5072
5073
5074
5075
5076
5077
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5078
5079
5080
5081
No return:  OSBC
5082
No return:  OSBCP
5083
5084
5085
5086
5087
5088
5089
5090
No return:  OTTR
5091
5092
5093
5094
5095
5096
5097
No return:  O

5466
5467
No return:  SIEN
5468
5469
5470
5471
5472
5473
5474
5475
5476
No return:  SIVB
5477
5478
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5479
5480
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5481
5482
5483
5484
5485
5486
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5487
5488
5489
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5490
5491
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5492
5493
5494
No return:  SMCP
5495
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5496
5497
5498
5499
No return:  SMTX
5500
It has been:  390.1166666666667  minutes
5501
5502
5503
5504
5505
5506
5507
5508
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5509
5510
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5511
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5512
No return:  SNY
5513
No return:  SOCL
5514
5515
5516
5517
5518
5519
5520
5521
5522
No return:  SORL
5523
<class 'Key

5931
No return:  ZBZZT
5932
No return:  ZCZZT
5933
5934
5935
No return:  ZFGN
5936
5937
5938
5939
5940
No return:  ZIONL
5941
5942
5943
5944
No return:  ZIONW
5945
5946
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
5947
5948
No return:  ZJZZT
5949
5950
5951
5952
5953
5954
5955
5956
No return:  ZVZZC
5957
No return:  ZVZZT
5958
Bad symbol in  ZXYZ.A
5959
No return:  ZXZZT
5960
5961
5962
5963
5964
5965
5966
No return:  AEF
5967
5968
5969
5970
No return:  AIRI
5971
5972
<class 'KeyError'> :  industry
<class 'KeyError'> :  sector
<class 'KeyError'> :  lastSplitDate
<class 'KeyError'> :  lastSplitFactor
5973
5974
5975
No return:  AMS
5976
5977
5978
5979
Bad symbol in  ARNC.PR
5980
5981
No return:  ATEST
5982
Bad symbol in  ATEST.A
5983
Bad symbol in  ATEST.B
5984
Bad symbol in  ATEST.C
5985
Bad symbol in  ATEST.G
5986
Bad symbol in  ATEST.H
5987
Bad symbol in  ATEST.L
5988
5989
5990
No return:  AUMN
5991
No return:  AWX
5992
5993
5994
5995
Bad symbol in  BCV.PR.A
5996
5997
599

In [13]:
stocks

,Symbol,Name,Exchange,industry,sector,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,upcoming_earnings_date,previous_earnings_date
0,A,"Agilent Technologies, Inc",NYSE,Diagnostics & Research,Healthcare,0.72,1593388800,1414972800,1398:1000,2020-11-23,2020-07-31 00:00:00
1,AA,Alcoa Corp,NYSE,Aluminum,Basic Materials,None,None,1477958400,1000:801,2020-10-14,2020-06-30 00:00:00
2,AAN,"Aaron's, Inc",NYSE,Rental & Leasing Services,Industrials,0.16,1600214400,1271376000,3:2,2020-11-02,2020-06-30 00:00:00
3,AAP,"Advance Auto Parts, Inc W/I",NYSE,Specialty Retail,Consumer Cyclical,1,1600300800,1127692800,3:2,2020-11-10,2020-07-11 00:00:00
4,AAT,"American Assets Trust, Inc",NYSE,REIT—Diversified,Real Estate,1,1599609600,None,None,2020-10-27,2020-06-30 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
6276,YCBD,"cbdMD, Inc.",AMEX,Drug Manufacturers—Specialty & Generic,Healthcare,None,None,None,None,2020-08-12,2020-06-30 00:00:00
6277,YCBD.PR.A,"cbdMD, Inc. 8.0% Series A Cumulative Convertib...",AMEX,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
6278,YUMA,"Yuma Energy, Inc",AMEX,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
6279,ZDGE,"Zedge, Inc",AMEX,Software—Application,Technology,None,None,None,None,2020-10-22,2020-04-30 00:00:00


In [15]:
stocks.to_csv('info.csv')

In [16]:
with open('bad_symbols.txt', 'w') as f:
    for s in bad_symbols:
        f.write(str(s) + "\n")
        
with open('symbols_no_return.txt', 'w') as f:
    for line in symbols_with_no_return:
        f.write(str(line) + "\n")